In [43]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [44]:
!pp install transformers

/bin/bash: pp: command not found


In [45]:
df= pd.read_csv('/kaggle/input/tweet-for-airline/Tweets.csv')

In [46]:
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [47]:
target_map = {'positive':1,'negative':0,'neutral':2}
df['target'] = df['airline_sentiment'].map(target_map)

In [48]:
df2 = df[['text','target']]

In [49]:
df2.head()

,text,target
0,@VirginAmerica What @dhepburn said.,2
1,@VirginAmerica plus you've added commercials t...,1
2,@VirginAmerica I didn't today... Must mean I n...,2
3,@VirginAmerica it's really aggressive to blast...,0
4,@VirginAmerica and it's a really big bad thing...,0


In [50]:
df2.columns = ['sentences','labels']

In [51]:
df2.head()

,sentences,labels
0,@VirginAmerica What @dhepburn said.,2
1,@VirginAmerica plus you've added commercials t...,1
2,@VirginAmerica I didn't today... Must mean I n...,2
3,@VirginAmerica it's really aggressive to blast...,0
4,@VirginAmerica and it's a really big bad thing...,0


In [52]:
df2.to_csv('my_data.csv',index =None)

In [53]:
!pip install datasets

In [54]:
from datasets import load_dataset

In [55]:
raw_dataset = load_dataset('csv',data_files = 'my_data.csv')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-62878d542673cdce/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [56]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['sentences', 'labels'],
        num_rows: 14640
    })
})

In [57]:
raw_dataset['train']

Dataset({
    features: ['sentences', 'labels'],
    num_rows: 14640
})

In [58]:
split = raw_dataset['train'].train_test_split(0.3,seed = 42)

In [59]:
split

DatasetDict({
    train: Dataset({
        features: ['sentences', 'labels'],
        num_rows: 10248
    })
    test: Dataset({
        features: ['sentences', 'labels'],
        num_rows: 4392
    })
})

In [60]:
from transformers import AutoTokenizer

In [61]:
checkpoint = "distilbert-base-cased"

In [62]:
# checkpoint = "distilbert-base-cased"
# tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# def tokenize_fn(batch):
#     return tokenizer(batch['sentences'],truncation =True)
# tokenized_datasets = split.map(tokenize_fn,batched  =True)
# model = AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels = 3)
# training_args = TrainingArguments(output_dir='training_dir',evaluation_strategy = 'epoch',save_strategy = 'epoch',num_train_epochs = 3,per_device_train_batch_size =16,per_device_eval_batch_size =32)
# trainer.train()

In [63]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-cased/snapshots/9d7568e4b20ed5db15ee30e99c7219bde9990762/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-cased",
  "activation": "gelu",
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.26.1",
  "vocab_size": 28996
}

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--distilbert-base-cased/snapshots/9d7568e4b20ed5db15ee30e99c7219bde9990762/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-cased/snapshots/9d7568e4b20ed5db15ee30e99c7219bde99

In [64]:
def tokenize_fn(batch):
    return tokenizer(batch['sentences'],truncation =True)

In [65]:
tokenized_datasets = split.map(tokenize_fn,batched  =True)

  0%|          | 0/11 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

In [66]:
from transformers import AutoModelForSequenceClassification,Trainer,TrainingArguments,AutoConfig


In [67]:
config = AutoConfig.from_pretrained(checkpoint)
config.id2label = {v:k for k,v in target_map.items()}

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-cased/snapshots/9d7568e4b20ed5db15ee30e99c7219bde9990762/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-cased",
  "activation": "gelu",
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.26.1",
  "vocab_size": 28996
}



In [69]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint,config=config)

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--distilbert-base-cased/snapshots/9d7568e4b20ed5db15ee30e99c7219bde9990762/pytorch_model.bin
Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForS

In [70]:
training_args = TrainingArguments(output_dir='training_dir',evaluation_strategy = 'epoch',save_strategy = 'epoch',num_train_epochs = 3,per_device_train_batch_size =16,per_device_eval_batch_size =32)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [71]:
def compute_metrics(logits_and_labels):
    logits,labels = logits_and_labels
    predictions = np.argmax(logits,axis=-1)
    acc = np.mean(predictions==labels) 
    return {"accuracy":acc}

In [72]:
trainer = Trainer(model,training_args,train_dataset=tokenized_datasets['train'],eval_dataset = tokenized_datasets['test'],tokenizer = tokenizer,compute_metrics = compute_metrics)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentences. If sentences are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 10248
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1923
  Number of trainable parameters = 65783811
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
You're using a Distil

Epoch,Training Loss,Validation Loss


In [ ]:
from transformers import pipeline 
savedmodel = pipeline("text-classification",model = "training_dir/checkpoint-1282",device=0)
split['test']
test_pred = savedmodel(split['test']['sentences'])

In [ ]:
test_pred

In [ ]:
def getlabel(d):
    return int(d['label'].split('_')[1])

In [ ]:
output_labels = [getlabel(d) for d in test_pred]

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
print("accuracy is ",accuracy_score(split['test']['labels'],output_labels) )

In [ ]:
s = split['test']['sentences'][0]
print(s)

In [ ]:
savedmodel(s)

In [ ]:
#we can use autoconfig to get positive instead of label1